In [6]:
import pandas as pd
import numpy as np
from util import read_plumed_file
from sklearn.preprocessing import StandardScaler

from msmbuilder.decomposition import tICA
from msmbuilder.msm import MarkovStateModel

#for paramater selection
from sklearn.pipeline import Pipeline
from msmbuilder.cluster import NDGrid
from sklearn.cross_validation import KFold

In [2]:
def tica_transform(
    df_list, 
    cvs, 
    lag_time=1,  # ns
    n_components=5,
    timestep=0.002,  # ns
    start_time=100,  # ns
    end_time=250,  # ns
    return_tica=False,
):
    start_time = 1000 * start_time  # convert from ns to ps
    end_time = 1000 * end_time  # convert from ns to ps
    
    # convert to number of steps to be used for tICA
    lag_time_steps = int(lag_time / timestep)
    
    ref = pd.concat(
        [df.loc[start_time:end_time, cvs] for df in df_list],
        ignore_index=True
    )

    scaler = StandardScaler()
    scaler.fit(ref.values)
    X = [scaler.transform(df.loc[start_time:end_time, cvs].values) for df in df_list]
    
    tica = tICA(n_components=n_components, lag_time=lag_time_steps)
    tica.fit(X)

    tica.summarize()
    if return_tica:
        return tica.transform(X), tica, scaler
    return tica.transform(X)

# Read in Data

In [8]:
iso_colvars = [
    read_plumed_file(f'../uremic_toxins/protein_bound/indoxyl_sulfate/unbiased/COLVAR.{idx}') for idx in range(1, 16)
]

### Get CVS

In [9]:
cvs_of_interest = np.append(iso_colvars[0].columns[0:-3:2],
                            iso_colvars[0].columns[-3:])
iso = pd.concat(
            [df.loc[50000:250000, cvs_of_interest] for df in iso_colvars],
            ignore_index=True
    )

### Testing Hyperparameters

In [19]:
n_states = [25,50,100,200]
lags=[.002,.005,.01,.015,.02]
comps=[2]
results = []
count=0
for c in comps:
    for l in lags:
        transformed_tica, tica, scaler = tica_transform(
            iso_colvars, 
            cvs_of_interest, 
            lag_time=l,
            n_components=c,
            start_time=50, 
            end_time=250, 
            return_tica=True,
            )
        maxs=[]
        mins=[]
        for i in transformed_tica:
            maxs.append(np.max(i))
            mins.append(np.min(i))
        grid_max=np.max(maxs)+1.2
        grid_min=np.min(mins)-1.2
        model = Pipeline([
            ('grid', NDGrid(min=-6, max=7)),
            ('msm', MarkovStateModel(n_timescales=4, lag_time=500, reversible_type='transpose', verbose=False))
        ])
        cv = KFold(len(transformed_tica), n_folds=5)
        for n in n_states:
            model.set_params(grid__n_bins_per_feature=n)
            for fold, (train_index, test_index) in enumerate(cv):
                train_data = [transformed_tica[i] for i in train_index]
                test_data = [transformed_tica[i] for i in test_index]
                model.fit(train_data)
                train_score = model.score(train_data)
                test_score = model.score(test_data)

                results.append({
                    'train_score': train_score,
                    'test_score': test_score,
                    'n_states': n,
                    'fold': fold,
                    'lag': l,
                    'components': c})
            print(results[count])
            count+=1

{'train_score': 4.5320933217074986, 'test_score': 3.0266217501661288, 'n_states': 25, 'fold': 0, 'lag': 0.002, 'components': 2}
{'train_score': 4.4847835299191567, 'test_score': 3.0901864737022788, 'n_states': 25, 'fold': 1, 'lag': 0.002, 'components': 2}
{'train_score': 3.9624918587158189, 'test_score': 3.0192458859663676, 'n_states': 25, 'fold': 2, 'lag': 0.002, 'components': 2}
{'train_score': 4.5292561542708967, 'test_score': 2.7284372930219529, 'n_states': 25, 'fold': 3, 'lag': 0.002, 'components': 2}
{'train_score': 4.525739076532461, 'test_score': 3.1167645383287566, 'n_states': 25, 'fold': 4, 'lag': 0.002, 'components': 2}
{'train_score': 4.5860597841447728, 'test_score': 3.0220110965977214, 'n_states': 50, 'fold': 0, 'lag': 0.002, 'components': 2}
{'train_score': 4.5448627250529547, 'test_score': 3.0435931215821181, 'n_states': 50, 'fold': 1, 'lag': 0.002, 'components': 2}
{'train_score': 4.1001354870473286, 'test_score': 3.1306986033139004, 'n_states': 50, 'fold': 2, 'lag': 0

In [20]:
results = pd.DataFrame(results)
results.head()

,components,fold,lag,n_states,test_score,train_score
0,2,0,0.002,25,3.026622,4.532093
1,2,1,0.002,25,3.090186,4.484784
2,2,2,0.002,25,3.019246,3.962492
3,2,3,0.002,25,2.728437,4.529256
4,2,4,0.002,25,3.116765,4.525739


In [21]:
#for grid check
avgs=results.groupby(['components','lag','n_states']).aggregate(np.mean).drop('fold', axis=1)
#to see result
results.groupby(['components','lag','n_states']).aggregate(np.mean).drop('fold', axis=1)

test_score  train_score
components lag   n_states                         
2          0.002 25          2.996251     4.406873
                 50          3.001230     4.478312
                 100         2.977239     4.501849
                 200         2.915010     4.516890
           0.005 25          2.999906     4.405618
                 50          3.011210     4.475481
                 100         2.980792     4.500337
                 200         2.908491     4.514807
           0.010 25          2.951973     4.396864
                 50          3.009616     4.470803
                 100         2.962057     4.496722
                 200         2.882702     4.512661
           0.015 25          2.971956     4.392822
                 50          2.988421     4.469739
                 100         2.966185     4.495520
                 200         2.875442     4.511381
           0.020 25          2.964305     4.387862
                 50          2.972512     4.466620
                 100         2.950941     4.492585
                 200         2.888661     4.508798

In [7]:
best_n = avgs['test_score'].argmax()
best_score = avgs.loc[best_n, 'test_score']
print(best_n, "states gives the best score:", best_score)

worst_n = avgs['test_score'].argmin()
worst_score = avgs.loc[worst_n, 'test_score']
print(worst_n, "states gives the worst score:", worst_score)

NameError: name 'avgs' is not defined

In [8]:
best_n=(2, 0.005, 50)